In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from pipelines import emission


from pipelines.nets.object_detection.yolo.brambox import boxes as bbb
from pipelines.nets.object_detection.yolo.brambox.boxes.util import visual as vis
from pipelines.nets.object_detection.yolo.brambox.boxes.detections.detection import *


In [80]:
import ast
def format_ultralytics_detections( img_dir, dets_dir = None, weights_cfg_file = '/media/kuser/datastore/nebula/pipelines/best_models/canis-majoris/cfg.json'):    
    with open('/media/kuser/datastore/nebula/pipelines/configs/entanglement_cfg.tml', 'r') as tmlfile:
        cfg = toml.load(tmlfile)
        
    with open(weights_cfg_file, 'r') as f:
        weights_cfg = json.load(f)
        weights_cfg['labels'] = [str(x) for x in ast.literal_eval(weights_cfg['labels'])]

        
    if not dets_dir:
        dets_dir = os.path.join(img_dir, 'annotations')
        os.makedirs(dets_dir, exist_ok = True)
    yolov3_dir = cfg['entanglement']['yolov3_dir']
    src_model_cfg = os.path.join(yolov3_dir, 'cfg', 'yolov3.cfg')
    src_data_cfg = os.path.join(yolov3_dir, 'cfg', 'yolov3.data')

    # Override model config
    dest_cfg_dir = os.path.abspath(os.path.join(dets_dir, os.pardir, 'cfg'))
    os.makedirs(dest_cfg_dir,exist_ok=True)
    yolo_cfg = format_model_config(weights_cfg, src_model_cfg, os.path.join(dest_cfg_dir, os.path.split(src_model_cfg)[-1]))

    current_data_cfg = parse_data_config(src_data_cfg)
    os.makedirs(os.path.join(dets_dir, 'data'), exist_ok = True)
    os.makedirs(os.path.join(dets_dir, 'cfg'), exist_ok = True)
    current_data_cfg = inject_data_config(dets_dir, current_data_cfg, tracking_classes)
    current_data_cfg_path = save_data_config(current_data_cfg, os.path.join(dets_dir, 'cfg', os.path.split(src_data_cfg)[-1]))

    
    # Run yolo to obtain detections
    emitter = emission.UltralyticsDetector(weights_file = weights_cfg['weights_path'], data_cfg = current_data_cfg_path, cfg = yolo_cfg, labels = weights_cfg['labels']  )
    dets = emitter.emit(img_dir, detection_threshold = 0.6)
    
    return dets

import urllib
import ntpath
from pipelineapi import PipelineAPI
import os
import json
import glob


def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


def parse_data_config(path):
        """Parses the data configuration file"""
        options = dict()
        with open(path, 'r') as fp:
            lines = fp.readlines()
        for line in lines:
            line = line.strip()
            if line == '' or line.startswith('#'):
                continue
            key, value = line.split('=')
            options[key.strip()] = value.strip()
        return options

    
def inject_data_config(dets_dir, data_config, tracking_classes):
    data_config['classes'] = len(tracking_classes)
    data_config['names'] = generate_names_cfg(dets_dir, tracking_classes)
    return data_config


def generate_names_cfg(dets_dir, tracking_classes):
    names_config = os.path.join(dets_dir, 'labels.names')
    with open(names_config, 'w+') as writer:
        for category in tracking_classes:
            writer.write(category.strip().replace('\'', '').replace('[', '').replace(']', '') +'\n')
    return names_config


def save_data_config(data_config, path, overwrite = False):
    """Saves the yolo-v3 data configuration file"""
    if not os.path.exists(path) or overwrite == True:
        with open(path, 'w') as writer:
            [writer.write(str(k)+'='+str(v)+'\n') for k,v in data_config.items()]
    return path


def format_model_config(weights_cfg, model_config_path, destination_cfg):
    anchors =str(weights_cfg['anchors']).replace('[','').replace(']','')
    dataset_category_names =weights_cfg['labels']


    content = None
    with open(model_config_path, 'r') as f:
        content = f.readlines()
        for idx, line in enumerate(content):
            key_vals = line.split('=')
            if len(key_vals) > 1:
                if 'anchors' in key_vals[0]:
                    key_vals[1] = str(anchors)
                    content[idx] = "=".join(key_vals)
                elif 'anchors' in key_vals[0]:
                    key_vals[1] = str(anchors)
                    content[idx] = "=".join(key_vals)
                elif 'classes' in key_vals[0]:
                    key_vals[1] = str(len(dataset_category_names))
                    content[idx] = "=".join(key_vals)
                elif 'filters' in key_vals[0] and ('yolo' in content[idx+2] or 'yolo' in content[idx+3] or 'yolo' in content[idx+4]):
                    key_vals[1] = str((len(dataset_category_names)+5)*3)
                    content[idx] = "=".join(key_vals)

                else:
                    for key, val in weights_cfg.items():
                        if len(key_vals) > 1 and str(key) in key_vals[0]:
                            key_vals[1] = str(val)
                            content[idx] = "=".join(key_vals)
                            break

    # Save model cfg #
    os.makedirs(os.path.dirname(destination_cfg), exist_ok = True)
    with open(destination_cfg, 'w') as f:        
        f.write("\n".join([c.strip("\n") for c in content]))
    return destination_cfg

In [ ]:
import os
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip
from moviepy.editor import CompositeVideoClip

from IPython.display import HTML
from PIL import Image
import shutil
import glob
import json
import toml
import cv2


yolo_python = '/home/kuser/miniconda2/envs/yolo/bin/python'
atm_dir = '~/atm/tools/'
cam_num = 1
extraction_dir = '/home/kuser/Desktop/demo_files/cam{}/'.format(cam_num)
save_dir = '/home/kuser/Desktop/demo_files/mp4cam{}/'.format(cam_num)

if not os.path.exists(extraction_dir):
    os.makedirs(extraction_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    


## Make to video ##
all_logdirs = os.listdir(extraction_dir)
with open('/media/kuser/datastore/nebula/pipelines/configs/entanglement_cfg.tml', 'r') as tmlfile:
    cfg = toml.load(tmlfile)

tracking_classes = cfg['entanglement']['deployed_classes']
idx2img = {}
class_label_map = {}
for idx, cat in enumerate(tracking_classes):
    class_label_map[cat] = idx
    
idx2cat = {v_i: v for v, v_i in class_label_map.items()}


for log_dir in all_logdirs:
    if os.path.isdir(os.path.join(extraction_dir, log_dir)):
        img_dir = os.path.join(extraction_dir, log_dir)
        
        # Run Detections
        yolo_dir = "/media/kuser/datastore/nebula/pipelines/nets/yolov3/"
        weights = os.path.join("/home/kuser/atm/nets/yolo/weights/yolov3-pronto-20190802.pt")
        anns_dir = os.path.join(os.path.abspath(img_dir), 'annotations')
        processed_dir = os.path.join(save_dir, log_dir, 'processed_frames')
        if True: # not os.path.exists(processed_dir):
            os.makedirs(processed_dir, exist_ok = True)

            dets = format_ultralytics_detections(img_dir)

            for uri, img_dets in dets:
                bramboxes = []
                for box in img_dets:
                    det = Detection()
                    det.x_top_left = int(box['box2d']['x1'])
                    det.y_top_left = int(box['box2d']['y1'])
                    det.width = int(box['box2d']['x2'])- int(box['box2d']['x1']) + 1
                    det.height = int(box['box2d']['y2'])- int(box['box2d']['y1']) + 1
                    det.confidence = box['confidence']
                    if class_label_map is not None:
                        det.class_label = box['category']
                    else:
                        det.class_label = str(int(idx2cat[box['category']]))

                    bramboxes.append(det)

                
                    img_name = path_leaf(uri)
                    img = cv2.imread(uri)
                    image = vis.draw_boxes(img, bramboxes, show_labels=True)
                    cv2.imwrite( f'{os.path.join(processed_dir, os.path.splitext(os.path.split(img_name)[-1])[0])}_detections.png', image)
                    

            # make video #
            log_video = "{}-cam{}.mp4".format(log_dir, cam_num)

            # Get Images
            exts = ['*.jpg', '*.png', '*.jpeg', '*.JPG']
            processed_imgs = [f for ext in exts for f in glob.glob(os.path.join(processed_dir, ext))] 
            if processed_imgs:
                clip = ImageSequenceClip(sorted(processed_imgs), fps = 30)
                clip.write_videofile(os.path.join(save_dir, log_video), audio=False)

                # Format Output Path
                HTML("""
                <video width="960" height="540" controls>
                 <source src="{0}">
                </video>
                """.format(save_dir))




# Get Frame Sequence
frame_sequence = []


Initializing Ultralytics Yolo Annotation Pipeline ... \o/ \o/ ...
           device0 _CudaDeviceProperties(name='GeForce GTX 1080', total_memory=8119MB)

image 1/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100416_cam1_lilyxavier-2019-08-02--13-51-42_1564753923.8974588_41.54589080810547--87.92728424072266.jpg: 352x416 13 cars, 1 traffic signs, 3 trucks, Done. (0.042s)
image 2/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100417_cam1_lilyxavier-2019-08-02--13-51-42_1564753923.9564588_41.54589080810547--87.92728424072266.jpg: 352x416 11 cars, 1 traffic signs, 3 trucks, Done. (0.036s)
image 3/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100418_cam1_lilyxavier-2019-08-02--13-51-42_1564753923.9964588_41.54589080810547--87.92728424072266.jpg: 352x416 11 cars, 1 traffic signs, 3 trucks, Done. (0.037s)
image 4/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100419_cam1_lilyxavier-2019-08-02--13-51-42_1564753924.0574586_41.54589080810547--87.92728424072266.jpg: 352x416 10 ca

image 37/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100452_cam1_lilyxavier-2019-08-02--13-51-42_1564753925.6954587_41.545955657958984--87.92698669433594.jpg: 352x416 12 cars, 2 traffic signs, 5 trucks, Done. (0.050s)
image 38/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100453_cam1_lilyxavier-2019-08-02--13-51-42_1564753925.7564588_41.54597854614258--87.92688751220703.jpg: 352x416 10 cars, 2 traffic signs, 4 trucks, Done. (0.040s)
image 39/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100454_cam1_lilyxavier-2019-08-02--13-51-42_1564753925.7964587_41.54597854614258--87.92688751220703.jpg: 352x416 11 cars, 2 traffic signs, 2 trucks, Done. (0.048s)
image 40/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100455_cam1_lilyxavier-2019-08-02--13-51-42_1564753925.8584588_41.54597854614258--87.92688751220703.jpg: 352x416 13 cars, 2 traffic signs, 3 trucks, Done. (0.045s)
image 41/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100456_cam1_lilyxavier-2019-08-02--13-51-42

image 75/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100490_cam1_lilyxavier-2019-08-02--13-51-42_1564753927.6934588_41.54604721069336--87.9266128540039.jpg: 352x416 12 cars, 2 traffic signs, 4 trucks, Done. (0.063s)
image 76/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100491_cam1_lilyxavier-2019-08-02--13-51-42_1564753927.7554588_41.54604721069336--87.9266128540039.jpg: 352x416 12 cars, 2 traffic signs, 4 trucks, Done. (0.045s)
image 77/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100492_cam1_lilyxavier-2019-08-02--13-51-42_1564753927.7954588_41.54607391357422--87.926513671875.jpg: 352x416 12 cars, 3 traffic signs, 3 trucks, Done. (0.046s)
image 78/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100493_cam1_lilyxavier-2019-08-02--13-51-42_1564753927.8554587_41.54607391357422--87.926513671875.jpg: 352x416 11 cars, 2 traffic signs, 4 trucks, Done. (0.048s)
image 79/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100494_cam1_lilyxavier-2019-08-02--13-51-42_156475

image 111/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100526_cam1_lilyxavier-2019-08-02--13-51-42_1564753929.5224588_41.54613494873047--87.92627716064453.jpg: 352x416 10 cars, 3 traffic signs, 2 trucks, Done. (0.042s)
image 112/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100527_cam1_lilyxavier-2019-08-02--13-51-42_1564753929.5634587_41.54613494873047--87.92627716064453.jpg: 352x416 9 cars, 3 traffic signs, 4 trucks, Done. (0.048s)
image 113/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100528_cam1_lilyxavier-2019-08-02--13-51-42_1564753929.6224587_41.54613494873047--87.92627716064453.jpg: 352x416 9 cars, 3 traffic signs, 2 trucks, Done. (0.034s)
image 114/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100529_cam1_lilyxavier-2019-08-02--13-51-42_1564753929.6674588_41.54613494873047--87.92627716064453.jpg: 352x416 9 cars, 3 traffic signs, 2 trucks, Done. (0.037s)
image 115/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100530_cam1_lilyxavier-2019-08-02--13-51-4

image 148/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100563_cam1_lilyxavier-2019-08-02--13-51-42_1564753931.3894587_41.54618835449219--87.92607879638672.jpg: 352x416 9 cars, 2 traffic signs, 1 trucks, Done. (0.043s)
image 149/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100564_cam1_lilyxavier-2019-08-02--13-51-42_1564753931.4284587_41.54620361328125--87.926025390625.jpg: 352x416 9 cars, 2 traffic signs, 2 trucks, Done. (0.041s)
image 150/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100565_cam1_lilyxavier-2019-08-02--13-51-42_1564753931.4924588_41.54620361328125--87.926025390625.jpg: 352x416 10 cars, 2 traffic signs, 2 trucks, Done. (0.037s)
image 151/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100566_cam1_lilyxavier-2019-08-02--13-51-42_1564753931.5324588_41.54620361328125--87.926025390625.jpg: 352x416 10 cars, 2 traffic signs, 2 trucks, Done. (0.045s)
image 152/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100567_cam1_lilyxavier-2019-08-02--13-51-42_156

...

image 338/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100753_cam1_lilyxavier-2019-08-02--13-51-42_1564753940.9904587_41.5463981628418--87.92535400390625.jpg: 352x416 14 cars, 7 trucks, Done. (0.060s)
image 339/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100754_cam1_lilyxavier-2019-08-02--13-51-42_1564753941.0284588_41.5463981628418--87.92535400390625.jpg: 352x416 13 cars, 7 trucks, Done. (0.043s)
image 340/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100755_cam1_lilyxavier-2019-08-02--13-51-42_1564753941.0884588_41.5463981628418--87.92535400390625.jpg: 352x416 15 cars, 5 trucks, Done. (0.045s)
image 341/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100756_cam1_lilyxavier-2019-08-02--13-51-42_1564753941.1274588_41.5463981628418--87.92535400390625.jpg: 352x416 17 cars, 5 trucks, Done. (0.050s)
image 342/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100757_cam1_lilyxavier-2019-08-02--13-51-42_1564753941.1894586_41.5463981628418--87.92535400390625.jpg: 352x416

image 377/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100792_cam1_lilyxavier-2019-08-02--13-51-42_1564753942.9284587_41.54643630981445--87.92522430419922.jpg: 352x416 9 cars, 6 trucks, Done. (0.038s)
image 378/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100793_cam1_lilyxavier-2019-08-02--13-51-42_1564753942.9894588_41.54643630981445--87.92522430419922.jpg: 352x416 11 cars, 5 trucks, Done. (0.044s)
image 379/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100794_cam1_lilyxavier-2019-08-02--13-51-42_1564753943.0304587_41.54643630981445--87.92522430419922.jpg: 352x416 11 cars, 5 trucks, Done. (0.040s)
image 380/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100795_cam1_lilyxavier-2019-08-02--13-51-42_1564753943.0884588_41.54643630981445--87.92522430419922.jpg: 352x416 9 cars, 5 trucks, Done. (0.036s)
image 381/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100796_cam1_lilyxavier-2019-08-02--13-51-42_1564753943.1264586_41.54643630981445--87.92522430419922.jpg: 352x

image 416/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100831_cam1_lilyxavier-2019-08-02--13-51-42_1564753944.9234588_41.54646682739258--87.92513275146484.jpg: 352x416 19 cars, 8 trucks, Done. (0.066s)
image 417/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100832_cam1_lilyxavier-2019-08-02--13-51-42_1564753944.9624586_41.54646682739258--87.92513275146484.jpg: 352x416 20 cars, 6 trucks, Done. (0.076s)
image 418/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100833_cam1_lilyxavier-2019-08-02--13-51-42_1564753945.0224588_41.54647445678711--87.92510223388672.jpg: 352x416 18 cars, 6 trucks, Done. (0.068s)
image 419/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100834_cam1_lilyxavier-2019-08-02--13-51-42_1564753945.0634587_41.54647445678711--87.92510223388672.jpg: 352x416 21 cars, 6 trucks, Done. (0.069s)
image 420/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100835_cam1_lilyxavier-2019-08-02--13-51-42_1564753945.1234586_41.54647445678711--87.92510223388672.jpg: 35

image 456/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100871_cam1_lilyxavier-2019-08-02--13-51-42_1564753946.9264588_41.54650115966797--87.92501831054688.jpg: 352x416 1 buss, 15 cars, 6 trucks, Done. (0.045s)
image 457/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100872_cam1_lilyxavier-2019-08-02--13-51-42_1564753946.9924588_41.54650115966797--87.92501831054688.jpg: 352x416 1 buss, 16 cars, 6 trucks, Done. (0.045s)
image 458/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100873_cam1_lilyxavier-2019-08-02--13-51-42_1564753947.0314586_41.54650115966797--87.92501831054688.jpg: 352x416 1 buss, 15 cars, 6 trucks, Done. (0.047s)
image 459/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100874_cam1_lilyxavier-2019-08-02--13-51-42_1564753947.0894587_41.54650115966797--87.92501831054688.jpg: 352x416 16 cars, 6 trucks, Done. (0.057s)
image 460/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100875_cam1_lilyxavier-2019-08-02--13-51-42_1564753947.1294587_41.5465087890625--87

image 495/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100910_cam1_lilyxavier-2019-08-02--13-51-42_1564753948.8894587_41.5465202331543--87.92495727539062.jpg: 352x416 23 cars, 3 trucks, Done. (0.051s)
image 496/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100911_cam1_lilyxavier-2019-08-02--13-51-42_1564753948.9284587_41.5465202331543--87.92495727539062.jpg: 352x416 23 cars, 3 trucks, Done. (0.050s)
image 497/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100912_cam1_lilyxavier-2019-08-02--13-51-42_1564753948.9894588_41.5465202331543--87.92495727539062.jpg: 352x416 23 cars, 3 trucks, Done. (0.050s)
image 498/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100913_cam1_lilyxavier-2019-08-02--13-51-42_1564753949.0554588_41.5465202331543--87.92495727539062.jpg: 352x416 21 cars, 3 trucks, Done. (0.055s)
image 499/512 /home/kuser/Desktop/demo_files/cam1/3last_part/100914_cam1_lilyxavier-2019-08-02--13-51-42_1564753949.0954587_41.5465202331543--87.92495727539062.jpg: 352x416

In [ ]:
# import os
# from moviepy.editor import VideoFileClip
# from moviepy.editor import ImageSequenceClip
# from moviepy.editor import CompositeVideoClip

# from IPython.display import HTML
# from PIL import Image
# import shutil
# import glob


# run_extraction = False
# # atm_python = '/home/dean/miniconda2/envs/atm/bin/python'
# yolo_python = '/home/kuser/miniconda2/envs/yolo/bin/python'

# # atm_dir = '~/atm/tools/'
# cam_num = 0
# extraction_dir = '/media/kuser/261f6d5a-186b-45ec-99eb-6cfb579cb0be/chinaVideosOutput/'
# save_dir = '/media/kuser/datastore/chinaConstructVideosOutput/'


# fps = 30

# if not os.path.exists(extraction_dir):
#     os.makedirs(extraction_dir)
    
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
    
# if run_extraction:
#     cmd = "{} {}dumpjpgs.py {} {} {} --sample_rate={}".format(atm_python, atm_dir, cam_num, logdir, extraction_dir, fps)
#     res = os.system(cmd)

# ## Make to video ##
# all_logdirs = os.listdir(extraction_dir)

# for log_dir in all_logdirs:
#     if os.path.isdir(os.path.join(extraction_dir, log_dir)):
#         img_dir = os.path.join(extraction_dir, log_dir)
        
#         # Run Detections
#         yolo_dir = "/media/kuser/datastore/nets/ObjectDetection-OneStageDet/yolo/"
#         weight = os.path.join(yolo_dir, "weights/construct-20190324.pt")
#         # anns_dir = os.path.join(os.path.abspath(img_dir), 'annotations')
#         processed_dir = os.path.join(extraction_dir, log_dir, 'processed_frames/')
#         if not os.path.exists(processed_dir):
#             os.makedirs(processed_dir)
#         construct_processed_dir = os.path.join(save_dir, log_dir,  'processed_construct_frames/')
#         if not os.path.exists(construct_processed_dir):
#             os.makedirs(construct_processed_dir)
            
#             print(os.path.abspath(img_dir))
#             print(save_dir)
#             print(processed_dir)

#             cmd = "cd {} && {} pipelines/detect_construct.py -csl -weight {} -image_dir {} -output_dir {}".format(yolo_dir, yolo_python, weight,processed_dir,  construct_processed_dir )
#             print(cmd)
#             res = os.system(cmd)



#             # make video #
#             log_video = "{}.mp4".format(log_dir)

#             # Get Images
#             exts = ['*.jpg', '*.png', '*.jpeg', '*.JPG']
#             processed_imgs = [f for ext in exts for f in glob.glob(os.path.join(construct_processed_dir, ext))] 
#             if processed_imgs:
#                 clip = ImageSequenceClip(sorted(processed_imgs), fps = fps)
#                 clip.write_videofile(os.path.join(save_dir, log_video), audio=False)

#                 # Format Output Path
#                 HTML("""
#                 <video width="960" height="540" controls>
#                  <source src="{0}">
#                 </video>
#                 """.format(save_dir))




# # Get Frame Sequence
# frame_sequence = []
